# PPatel VacationPy


In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
weather_data = 'weatherdata.csv'
cities_data = pd.read_csv(weather_data)
cities_data.head()

,city,country,lat,lon,temp,humidity,cloudiness,windspeed
0,san cristobal,VE,7.77,-72.22,84.20,54,20,8.05
1,ola,RU,59.58,151.28,24.44,93,80,19.44
2,san patricio,MX,19.22,-104.70,82.40,65,5,17.22
3,takayama,JP,36.13,137.25,32.86,66,90,3.42
4,corralillo,CU,22.98,-80.59,75.16,71,0,17.20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
gmaps.configure(api_key=g_key)

In [25]:
humidity = cities_data['humidity']
locations = cities_data[['lat', 'lon']]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
ideal_hum = cities_data.loc[cities_data["humidity"] > 80]
ideal_humcloud = ideal_hum.loc[ideal_hum["cloudiness"] <= 50]
ideal_humcloudwind = ideal_humcloud.loc[ideal_humcloud["windspeed"] < 15]
ideal_humcloudwind_lowtemp = ideal_humcloudwind.loc[ideal_humcloudwind["temp"] >= 70]             
ideal_conditions = ideal_humcloudwind_lowtemp.loc[ideal_humcloudwind_lowtemp["temp"] < 90]
ideal_conditions.count()

city          33
country       33
lat           33
lon           33
temp          33
humidity      33
cloudiness    33
windspeed     33
dtype: int64

In [27]:
ideal_conditions.head()

,city,country,lat,lon,temp,humidity,cloudiness,windspeed
34,mar del plata,AR,-38.00,-57.56,71.74,88,44,11.41
45,le port,RE,-20.94,55.29,71.35,83,40,8.05
65,vila velha,BR,-20.33,-40.29,78.06,83,20,6.93
98,georgetown,MY,5.41,100.34,78.40,94,20,3.36
150,nampula,MZ,-15.12,39.27,71.60,94,40,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
hotel_df = pd.DataFrame(ideal_conditions)
hotel_df["hotel"] = ""
hotel_df.head()

,city,country,lat,lon,temp,humidity,cloudiness,windspeed,hotel
34,mar del plata,AR,-38.00,-57.56,71.74,88,44,11.41,
45,le port,RE,-20.94,55.29,71.35,83,40,8.05,
65,vila velha,BR,-20.33,-40.29,78.06,83,20,6.93,
98,georgetown,MY,5.41,100.34,78.40,94,20,3.36,
150,nampula,MZ,-15.12,39.27,71.60,94,40,6.93,


In [29]:
for index, row in hotel_df.iterrows():
    lat = hotel_df.loc[index, "lat"]
    lng = hotel_df.loc[index, "lon"]
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    target_coordinates = f"{lat},{lng}"
    target_search = "hotel"
    target_radius = 8000
    target_type = "hotel"

    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}
    
    hotels = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "hotel"] = hotels['results'][0]['name']
    except:
        print("no hotels available")

no hotels available
no hotels available


In [30]:
hotel_df.head()

,city,country,lat,lon,temp,humidity,cloudiness,windspeed,hotel
34,mar del plata,AR,-38.00,-57.56,71.74,88,44,11.41,Hermitage Hotel
45,le port,RE,-20.94,55.29,71.35,83,40,8.05,Lodge Roche Tamarin & Spa Village Nature
65,vila velha,BR,-20.33,-40.29,78.06,83,20,6.93,Ibis sleep inn
98,georgetown,MY,5.41,100.34,78.40,94,20,3.36,Arte S Penang
150,nampula,MZ,-15.12,39.27,71.60,94,40,6.93,Hotel Paraíso


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel}</dd>
<dt>City</dt><dd>{city}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [70]:
# Add marker layer ontop of heat map
hotel = hotel_df["hotel"]
city = hotel_df["city"]
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel Suggestion: {hotel}" for hotel in hotel])

fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>